# Arbeit mit Selenium

Die Arbeit mit Selenium erfordert etwas Übung. Aber der Zeitaufwand lohnt sich. Es gibt mit Selenium kaum ein Webdienst der nicht scrapbar wird. Beginnen wir aber wie üblich mit der Dokumentation. Sie ist im Falle von Selenium sehr hilfreich. Ihr findet [sie hier](http://selenium-python.readthedocs.io/). Und [hier](http://selenium-python.readthedocs.io/locating-elements.html). 

Um Selenium kennenzulernen, gehen wir zurück zu unserem Beispiel der Lehren: https://www.berufsberatung.ch/dyn/show/2930. Nun wollen wir keine URLs generieren, um unsere Inhalte zu finden. Wir wollen stattdessen mit der Site interagieren. So sollten wir alle Einträge bekommen. BeautifulSoup werden wir trotzdem noch dazu nehmen. Denn Selenium liest keine Inhalte aus. Die Library lässt uns einfach mit dem Webdienst interagieren.

Beginnen wir mit den Imports

In [3]:
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

Und dann schicken wir Selenium auf die Seite.

In [4]:
#Wir starten den Browser:
driver = webdriver.Chrome()
#Wir besuchen die Site
driver.get("https://www.berufsberatung.ch/dyn/show/2930")

Nun suchen wir mit dem Inspector die Elemente, die wir ansteuern wollen.

In [5]:
driver.find_element_by_class_name("fs-autocomplete-trigger").click()

In [7]:
driver.find_element_by_id("sw_453").click()

ElementNotVisibleException: Message: element not interactable
  (Session info: chrome=69.0.3497.100)
  (Driver info: chromedriver=2.42.591059 (a3d9684d10d61aa0c45f6723b327283be1ebaad8),platform=Mac OS X 10.12.6 x86_64)


In [8]:
driver.find_element_by_id("uxfs-action").click()

In [14]:
test = driver.page_source
test

'<!DOCTYPE html><!--[if IE 7]> \t\t\t\t <html class="no-js ie7 mobileVersion" lang="de"> <![endif]--><!--[if IE 8]> \t\t\t\t <html class="no-js mobileVersion lt-ie9" lang="de"> <![endif]--><!--[if IE 9]>\t\t\t \t\t<html class="no-js mobileVersion ie9" lang="de"> <![endif]--><!--[if gt IE 8]>\t\t\t <html class="no-js mobileVersion" lang="de"> <![endif]--><html xmlns="http://www.w3.org/1999/xhtml" lang="de" class="js no-touch cssanimations csstransitions chrome" style=""><head>\n<meta charset="utf-8" />\n<meta name="publisher" content="SDBB | CSFO" />\n<meta name="distribution" content="global" />\n<meta name="author" content="SDBB | CSFO" />\n<meta name="copyright" content="SDBB | CSFO" />\n<meta name="rating" content="general" />\n<meta name="language" content="de" />\n<meta id="viewport" name="viewport" content="width=device-width, initial-scale=1" />\n<meta name="cludocrawler" content="include" />\n<title>Suche Lehrstelle - berufsberatung.ch</title>\n<link rel="apple-touch-icon" size

In [17]:
# Wir öffnen eine Datei zum Schreiben ("w": write)
file = open("lehrstellen.html", "w")
file.write(test)

# Abschließend müssen wir die Datei wieder schließen
file.close()

Aber wir wollen alle Ergänzen. Holen wir deshalb die Nummern nochmals

In [19]:
r = requests.get('https://www.berufsberatung.ch/dyn/show/2930')
soup = BeautifulSoup(r.text, 'html.parser')
ids = []
for elem in soup.find('ul',{'class':'ui-autocomplete full-list '}).find_all('a'):
    elem = "sw_" + elem['data-id']
    ids.append(elem)

Testen wir es mit den ersten fünf Einträgen

In [22]:
for elem in ids[:5]:
    print(elem)
    time.sleep(.5) #damit es nicht zu schnell geht
    driver.find_element_by_class_name("fs-autocomplete-trigger").click()
    time.sleep(.5)
    driver.find_element_by_id(elem).click()

sw_453
sw_452
sw_55
sw_7617
sw_7618


In [23]:
driver.find_element_by_id("uxfs-action").click()

Zeigen wird alle Ergebnisse an

In [24]:
driver.find_element_by_id("aSearchPaging").click()

Speichern wir die Ergebnisse ab

In [25]:
text = driver.page_source

In [28]:
def lehrstellen(html):
    
    soup = BeautifulSoup(html, 'html.parser')
    
    ortsliste = soup.find('div', {'class':'resultpart result-body'})\ #Backslash ist um Zeilenumbruch zu maskieren
                    .find_all('div', {'class':'display-table-cell table-col-3'})
    
    firmenliste = soup.find('div', {'class':'resultpart result-body'})\
                    .find_all('div', {'class':'display-table-cell bold company data-id table-col-1'})
    
    jahresliste = soup.find('div', {'class':'resultpart result-body'})\
                    .find_all('div', {'class':'display-table-cell float-left-for-sd table-col-4'})
        
    anzahlliste = soup.find('div', {'class':'resultpart result-body'})\
                    .find_all('div', {'class':'display-table-cell text-align-center float-left-for-sd table-col-5'})
    
    lst = []
    
    for ort, firma, jahr, anzahl in zip(ortsliste,firmenliste,jahresliste, anzahlliste):
        
        mini_dict = {'Ort':ort.text,
                     'Firma':firma.text,
                     'Jahr':jahr.text,
                     'Anzahl':int(anzahl.text.replace(' Lehrstelle(n)\n','').replace('\n',''))}
        lst.append(mini_dict)
    
    return lst

In [29]:
lehrstellen(text)

[{'Ort': 'Aadorf (TG)',
  'Firma': 'Jakob Tanner AG',
  'Jahr': '2019',
  'Anzahl': 1},
 {'Ort': 'Aesch BL (BL)',
  'Firma': 'peressini roofing AG',
  'Jahr': '2019',
  'Anzahl': 1},
 {'Ort': 'Alpnach Dorf (OW)',
  'Firma': 'Abdichtungsbau Durrer AG',
  'Jahr': '2019',
  'Anzahl': 1},
 {'Ort': 'Altendorf (SZ)',
  'Firma': 'Marty Stefan AG',
  'Jahr': '2019',
  'Anzahl': 1},
 {'Ort': 'Altstätten SG (SG)',
  'Firma': 'Arthur Müggler & Co. AG',
  'Jahr': '2019',
  'Anzahl': 1},
 {'Ort': 'Altstätten SG (SG)',
  'Firma': 'Räss AG',
  'Jahr': '2019',
  'Anzahl': 1},
 {'Ort': 'Amriswil (TG)',
  'Firma': 'Vogel Dach- und Fassadenbau AG',
  'Jahr': '2019',
  'Anzahl': 1},
 {'Ort': 'Amriswil (TG)',
  'Firma': 'Weber Bedachungen',
  'Jahr': '2019',
  'Anzahl': 1},
 {'Ort': 'Bad Ragaz (SG)',
  'Firma': 'Bürer Flachdach AG',
  'Jahr': '2019',
  'Anzahl': 1},
 {'Ort': 'Bassersdorf (ZH)',
  'Firma': 'Carl Meier Sohn AG',
  'Jahr': '2019',
  'Anzahl': 1},
 {'Ort': 'Bätterkinden (BE)',
  'Firma': 'E. J

## Bringen wir alles zusammen

In [ ]:
#Funktion, um nur die Informationen herauszuziehen, die uns interessieren
def lehrstellen(html):
    
    soup = BeautifulSoup(html, 'lxml')
    try:
        ortsliste = soup.find('div', {'class':'resultpart result-body'})\
                    .find_all('div', {'class':'display-table-cell table-col-3'})
    
        firmenliste = soup.find('div', {'class':'resultpart result-body'})\
                    .find_all('div', {'class':'display-table-cell bold company data-id table-col-1'})
    
        jahresliste = soup.find('div', {'class':'resultpart result-body'})\
                    .find_all('div', {'class':'display-table-cell float-left-for-sd table-col-4'})
        
        anzahlliste = soup.find('div', {'class':'resultpart result-body'})\
                    .find_all('div', {'class':'display-table-cell text-align-center float-left-for-sd table-col-5'})
    
        lehrstelle = soup.find('ul',{'class':'ui-autocomplete full-list '})\
                    .find_all('a')
        lst = []
    
        for ort, firma, jahr, anzahl,lehr in zip(ortsliste,firmenliste,jahresliste, anzahlliste,lehrstelle):
        
            mini_dict = {'Ort':ort.text,
                     'Firma':firma.text,
                     'Jahr':jahr.text,
                     'Anzahl':int(anzahl.text.replace(' Lehrstelle(n)\n','').replace('\n','')),
                     'Lehrstelle':lehr['data-value']}
            lst.append(mini_dict)
    
        return pd.DataFrame(lst).to_csv("d/"+str(datetime.datetime.now())+".csv")
    
    except:
        return pd.DataFrame([{'Ort':'Keine Treffer',
                'Firma':'Keine Treffer',
                'Jahr':'Keine Treffer',
                'Anzahl':'Keine Treffer'}])
    
#Bauen wir Listen aller Job-IDs
r = requests.get('https://www.berufsberatung.ch/dyn/show/2930')
soup = BeautifulSoup(r.text, 'lxml')
ids = []
for elem in soup.find('ul',{'class':'ui-autocomplete full-list '}).find_all('a'):
    elem = "sw_" + elem['data-id']
    ids.append(elem)
    
#Teilen wir diese Listen mit Länge von je 5 Teilen. 
#Das habe ich nicht selber geschrieben, sondern hier geholt:
#https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
idslst = [ids[i:i + 5] for i in range(0, len(ids), 5)]

count = 0
for ids in idslst:

    #Starten wir den Chrome-Browser und besuchen die Site
    driver = webdriver.Chrome('/usr/local/bin/chromedriver')
    driver.get("https://www.berufsberatung.ch/dyn/show/2930")

    #Bereiten wir die Suche vor    
    for elem in ids:
        time.sleep(1) #damit es nicht zu schnell geht
        driver.find_element_by_class_name("fs-autocomplete-trigger").click()
        time.sleep(1)
        driver.find_element_by_id(elem).click()
    
    #Suchen wir
    time.sleep(1)
    driver.find_element_by_id("uxfs-action").click()

    #Nun nun sorgen wir dafür, dass alle Ergebnisse anzeigt werden.
    exists = 1
    while(exists==1):
        loadmore = driver.find_element_by_id("aSearchPaging")
        if loadmore.text == "MEHR ERGEBNISSE ANZEIGEN":
            driver.find_element_by_id("aSearchPaging").click()
            time.sleep(1)
        else:
            exists = 0
            
    print(count)
    count += 1
    
    lehrstellen(driver.page_source)
    driver.close()

Kreieren wir ein kleine .py File und lösen es von der Commandline aus.